In [1]:
import numpy as np
from metpy.calc import relative_humidity_from_dewpoint
from metpy.units import units
import xarray as xr
import os.path
import sys
# Need to append the path to import the regrid function
sys.path.append('C:/Users/ophme/Documents/wildfire-repo/climate-process/climate-calc/')
from regrid import *
import glob
import xarray as xr
import numpy as np
from dask.distributed import LocalCluster
from ibicus.debias import LinearScaling, DeltaChange, QuantileMapping, CDFt, ISIMIP, ScaledDistributionMapping
import time
import dask.array as da 
import os

In [2]:
# Used for visualisation purposes
client = LocalCluster().get_client()

C:\Users\ophme\AppData\Local\anaconda3\envs\wildfire\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53649 instead
  warnings.warn(


In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:53649/status,
Dashboard: http://127.0.0.1:53649/status,Workers: 4
Total threads: 12,Total memory: 31.65 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53650,Workers: 4
Dashboard: http://127.0.0.1:53649/status,Total threads: 12
Started: Just now,Total memory: 31.65 GiB
Comm: tcp://127.0.0.1:53676,Total threads: 3
Dashboard: http://127.0.0.1:53679/status,Memory: 7.91 GiB
Nanny: tcp://127.0.0.1:53653,


### 1. Regrid

#### Open files for regridding
A temperature file from ERA5-land and from Climex is used here to create the regridder. 

The files are not needed afterwards.

In [4]:
path = "O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/CLIMEX2/fwi/eoo/"
files = glob.glob(path + '*')
files = sorted(files)[:1] # Only 1 files as this is just for regridding purposes 
#datasets = [xr.open_dataset(file_path).sel(rlat = slice(rlat1, rlat2), rlon = slice(-(360-rlon1), -(360-rlon2))) for file_path in files]
ds_out = xr.open_dataset('O:/Public/sharing-4270-CERM/VLYMI/CLIMEX2/GlobusDownload/ClimExII_4_Friends/ClimExII_4_Friends/sftof_eoo_206400_se.nc')

In [5]:
ds_out

<xarray.Dataset> Size: 5MB
Dimensions:       (height: 1, rlat: 412, rlon: 460, time: 1)
Coordinates:
  * height        (height) float64 8B 0.0
    lat           (rlat, rlon) float64 2MB ...
    lon           (rlat, rlon) float64 2MB ...
  * rlat          (rlat) float64 3kB -23.38 -23.27 -23.16 ... 21.61 21.73 21.83
  * rlon          (rlon) float64 4kB -31.67 -31.56 -31.45 ... 18.6 18.71 18.82
  * time          (time) datetime64[ns] 8B 2064-01-01
Data variables:
    rotated_pole  |S1 1B ...
    sftof         (rlat, rlon) float64 2MB ...
Attributes: (12/22)
    Conventions:                    CF-1.6
    contact:                        giguere.michel@ouranos.ca
    comment:                        CRCM5 v3331 0.22 deg AMNO22d2 L56 S17-15m...
    creation_date:                  2020-08-04 
    experiment:                     simulation de reoorence 
    experiment_id:                  eoo 
    ...                             ...
    ouranos_run_id:                 eoo OURALIB 1.5
    product:                        output
    reoorence:                      http://www.ouranos.ca
    history:                        Thu Dec 22 15:48:10 2022: ncatted -a long...
    NCO:                            netCDF Operators version 4.9.3 (Homepage ...
    history_of_appended_files:      Thu Dec 22 15:48:10 2022: Appended file /...

In [6]:
path =  "O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/unprocess/FWI-variables/Europe/ERA5-land/"
files = glob.glob(path + '*')
files = sorted(files)[2] # Only 1 files as this is just for regridding purposes 

ds_in = xr.open_dataset(files, engine = "cfgrib")
ds_in = ds_in.rename({'latitude': 'lat','longitude': 'lon'})

Ignoring index file 'O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/unprocess/FWI-variables/Europe/ERA5-land\\1980-01.grib.5b7b6.idx' incompatible with GRIB file


In [7]:
# Create regridder with ERA5 as input grid and Climex as output grid
regridder = regridder_masked(ds_in, ds_out, 't2m')

751 752
451 452
189520 189980
189520 189980


In [8]:
regridder

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_451x751_412x460.nc 
Reuse pre-computed weights? False 
Input grid shape:           (451, 751) 
Output grid shape:          (412, 460) 
Periodic in longitude?      False

In [9]:
del ds_in
del ds_out

### 2. Calculate FWI variables (temp, wind, rhum and prcp) and regrid 

#### Temperature

In [11]:
Case_Study = False # If False, the whole European domain is downloaded 

CS_name = 'Crete' 

start_mid_year = False
home_path = "O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/unprocess/FWI-variables/"

if Case_Study == True:
    if CS_name == 'Crete':
        path = home_path + "Case_Studies/Crete/"
    else:
        print('CS not known')
else:
    path = home_path + "Europe/"

years = np.arange(1991, 2011, 1)

months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

for year in years:
    print(year)
    year = str(year)

    for m, month in enumerate(months): 
        print(month)
        fname = path + "/Derived-FWI-ERA5-land/vars-" + year + "-" + month + ".nc"

        # Only download files which are not in the folder
        if os.path.isfile("O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/regrided/FWI-variables/temp-" + year + "-" + month + ".nc") == True:
            continue
            
        #ds = xr.open_dataset(path + "ERA5-land/" + year + "-" + month + "-prcp.grib", engine = "cfgrib") # load the prcp at noon and midnight
        ds1 = xr.open_dataset(path  + "ERA5-land/" +  year + "-" + month + ".grib", engine = "cfgrib") # load the temp, dew temp and wind at noon
        
        #ds1 = ds1.assign(prcp=(['time','latitude','longitude'], ds.isel(step = 1, time = slice(0, len(ds.time)-1)).tp.data)) # m to mm 
 
        ds1 = ds1.assign(temp=lambda x: x.t2m - 273.15) # Convert K to C
 
       #ds1 = ds1.assign(wind=lambda x: np.sqrt(x.u10*x.u10 + x.v10*x.v10)*0.001*3600) # Calculate wind speed in km/h
   
        #ds1 = ds1.assign(rhum=lambda x: relative_humidity_from_dewpoint(x.temp* units.degC, (x.d2m- 273.15)* units.degC )*100) # Calculate relative humidity [%]

        ds1 = ds1.drop_vars(['u10', 'v10', 'd2m', 't2m', 'step'])
        ds1 = ds1.rename({'latitude': 'lat','longitude': 'lon'})
        ds1 = regridder(ds1, skipna=True, na_thres=0.5)
        ds1.to_netcdf("O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/regrided/FWI-variables/temp-" + year + "-" + month + ".nc")


1991
01
02
03
04
05
06
07
08
09
10
11
12
1992
01
02
03
04
05
06
07
08
09
10
11
12
1993
01
02
03
04
05
06
07
08
09
10
11
12
1994
01
02
03
04
05
06
07
08
09
10
11
12
1995
01
02
03
04
05
06
07
08
09
10
11
12
1996
01
02
03
04
05
06
07
08
09
10
11
12
1997
01
02
03
04
05
06
07
08
09
10
11
12
1998
01
02
03
04
05
06
07
08
09
10
11
12
1999
01
02
03
04
05
06
07
08
09
10
11
12
2000
01
02
03
04
05
06
07
08
09
10
11
12
2001
01
02
03
04
05
06
07
08
09
10
11
12
2002
01
02
03
04
05
06
07
08
09
10
11
12
2003
01
02
03
04
05
06
07
08
09
10
11
12
2004
01
02
03
04
05
06
07
08
09
10
11
12
2005
01
02
03
04
05
06
07
08
09
10
11
12
2006
01
02
03
04
05
06
07
08
09
10
11
12
2007
01
02
03
04
05
06
07
08
09
10
11
12
2008
01
02
03
04
05
06
07
08
09
10
11
12
2009
01
02
03
04
05
06
07
08
09
10
11
12
2010
01
02
03
04
05
06
07
08
09
10
11
12


#### Wind

In [12]:


Case_Study = False # If False, the whole European domain is downloaded 

CS_name = 'Crete' 

start_mid_year = False
home_path = "O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/unprocess/FWI-variables/"

if Case_Study == True:
    if CS_name == 'Crete':
        path = home_path + "Case_Studies/Crete/"
    else:
        print('CS not known')
else:
    path = home_path + "Europe/"

years = np.arange(1991, 2011, 1)

months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

for year in years:
    print(year)
    year = str(year)

    for m, month in enumerate(months): 
        print(month)
        fname = path + "/Derived-FWI-ERA5-land/vars-" + year + "-" + month + ".nc"

        # Only download files which are not in the folder
        if os.path.isfile("O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/regrided/FWI-variables/wind-" + year + "-" + month + ".nc") == True:
            continue
            
        #ds = xr.open_dataset(path + "ERA5-land/" + year + "-" + month + "-prcp.grib", engine = "cfgrib") # load the prcp at noon and midnight
        ds1 = xr.open_dataset(path  + "ERA5-land/" +  year + "-" + month + ".grib", engine = "cfgrib") # load the temp, dew temp and wind at noon
        
        #ds1 = ds1.assign(prcp=(['time','latitude','longitude'], ds.isel(step = 1, time = slice(0, len(ds.time)-1)).tp.data)) # m to mm 
 
        #ds1 = ds1.assign(temp=lambda x: x.t2m - 273.15) # Convert K to C
 
        ds1 = ds1.assign(wind=lambda x: np.sqrt(x.u10*x.u10 + x.v10*x.v10)*0.001*3600) # Calculate wind speed in km/h
   
        #ds1 = ds1.assign(rhum=lambda x: relative_humidity_from_dewpoint(x.temp* units.degC, (x.d2m- 273.15)* units.degC )*100) # Calculate relative humidity [%]

        ds1 = ds1.drop_vars(['u10', 'v10', 'd2m', 't2m', 'step'])
        ds1 = ds1.rename({'latitude': 'lat','longitude': 'lon'})
        ds1 = regridder(ds1, skipna=True, na_thres=0.5)
        ds1.to_netcdf("O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/regrided/FWI-variables/wind-" + year + "-" + month + ".nc")


1991
01
02
03
04
05
06
07
08
09
10
11
12
1992
01
02
03
04
05
06
07
08
09
10
11
12
1993
01
02
03
04
05
06
07
08
09
10
11
12
1994
01
02
03
04
05
06
07
08
09
10
11
12
1995
01
02
03
04
05
06
07
08
09
10
11
12
1996
01
02
03
04
05
06
07
08
09
10
11
12
1997
01
02
03
04
05
06
07
08
09
10
11
12
1998
01
02
03
04
05
06
07
08
09
10
11
12
1999
01
02
03
04
05
06
07
08
09
10
11
12
2000
01
02
03
04
05
06
07
08
09
10
11
12
2001
01
02
03
04
05
06
07
08
09
10
11
12
2002
01
02
03
04
05
06
07
08
09
10
11
12
2003
01
02
03
04
05
06
07
08
09
10
11
12
2004
01
02
03
04
05
06
07
08
09
10
11
12
2005
01
02
03
04
05
06
07
08
09
10
11
12
2006
01
02
03
04
05
06
07
08
09
10
11
12
2007
01
02
03
04
05
06
07
08
09
10
11
12
2008
01
02
03
04
05
06
07
08
09
10
11
12
2009
01
02
03
04
05
06
07
08
09
10
11
12
2010
01
02
03
04
05
06
07
08
09
10
11
12


#### Relative humidity

In [10]:
Case_Study = False # If False, the whole European domain is downloaded 

CS_name = 'Crete' 

start_mid_year = False
home_path = "O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/unprocess/FWI-variables/"

if Case_Study == True:
    if CS_name == 'Crete':
        path = home_path + "Case_Studies/Crete/"
    else:
        print('CS not known')
else:
    path = home_path + "Europe/"

years = np.arange(1991, 2011, 1)

months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

for year in years:
    print(year)
    year = str(year)

    for m, month in enumerate(months): 
        print(month)
        fname = path + "/Derived-FWI-ERA5-land/vars-" + year + "-" + month + ".nc"

        # Only download files which are not in the folder
        if os.path.isfile("O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/regrided/FWI-variables/rhum-" + year + "-" + month + ".nc") == True:
            continue
            
        #ds = xr.open_dataset(path + "ERA5-land/" + year + "-" + month + "-prcp.grib", engine = "cfgrib") # load the prcp at noon and midnight
        ds1 = xr.open_dataset(path  + "ERA5-land/" +  year + "-" + month + ".grib", engine = "cfgrib") # load the temp, dew temp and wind at noon
        
        #ds1 = ds1.assign(prcp=(['time','latitude','longitude'], ds.isel(step = 1, time = slice(0, len(ds.time)-1)).tp.data)) # m to mm 
 
        ds1 = ds1.assign(temp=lambda x: x.t2m - 273.15) # Convert K to C
 
        #ds1 = ds1.assign(wind=lambda x: np.sqrt(x.u10*x.u10 + x.v10*x.v10)*0.001*3600) # Calculate wind speed in km/h
   
        ds1 = ds1.assign(rhum=lambda x: relative_humidity_from_dewpoint(x.temp* units.degC, (x.d2m- 273.15)* units.degC )*100) # Calculate relative humidity [%]
    
        ds1 = ds1.drop_vars(['u10', 'v10', 'd2m', 't2m', 'step', 'temp'])
        ds1 = ds1.assign(rhum=(['time','latitude','longitude'], ds1.rhum.values)) 
        ds1 = ds1.rename({'latitude': 'lat','longitude': 'lon'})
        ds1 = regridder(ds1, skipna=True, na_thres=0.5)
        ds1.to_netcdf("O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/regrided/FWI-variables/rhum-" + year + "-" + month + ".nc")


1991
01
02
03
04
05
06
07
08
09
10
11
12
1992
01
02
03
04
05
06
07
08
09
10
11
12
1993
01
02
03
04
05
06
07
08
09
10
11
12
1994
01
02
03
04
05
06
07
08
09
10
11
12
1995
01
02
03
04
05
06
07
08
09
10
11
12
1996
01
02
03
04
05
06
07
08
09
10
11
12
1997
01
02
03
04
05
06
07
08
09
10
11
12
1998
01
02
03
04
05
06
07
08
09
10
11
12
1999
01
02
03
04
05
06
07
08
09
10
11
12
2000
01
02
03
04
05
06
07
08
09
10
11
12
2001
01
02
03
04


Ignoring index file 'O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/unprocess/FWI-variables/Europe/ERA5-land/2001-05.grib.5b7b6.idx' incompatible with GRIB file


05


C:\Users\ophme\AppData\Local\anaconda3\envs\wildfire\Lib\site-packages\xesmf\frontend.py:718: UserWarning: Using dimensions ('lat', 'lon') from data variable rhum as the horizontal dimensions for the regridding.
  warnings.warn(
Ignoring index file 'O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/unprocess/FWI-variables/Europe/ERA5-land/2001-06.grib.5b7b6.idx' incompatible with GRIB file


06


...

C:\Users\ophme\AppData\Local\anaconda3\envs\wildfire\Lib\site-packages\xesmf\frontend.py:718: UserWarning: Using dimensions ('lat', 'lon') from data variable rhum as the horizontal dimensions for the regridding.
  warnings.warn(
Ignoring index file 'O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/unprocess/FWI-variables/Europe/ERA5-land/2010-11.grib.5b7b6.idx' incompatible with GRIB file


11


C:\Users\ophme\AppData\Local\anaconda3\envs\wildfire\Lib\site-packages\xesmf\frontend.py:718: UserWarning: Using dimensions ('lat', 'lon') from data variable rhum as the horizontal dimensions for the regridding.
  warnings.warn(
Ignoring index file 'O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/unprocess/FWI-variables/Europe/ERA5-land/2010-12.grib.5b7b6.idx' incompatible with GRIB file


12


C:\Users\ophme\AppData\Local\anaconda3\envs\wildfire\Lib\site-packages\xesmf\frontend.py:718: UserWarning: Using dimensions ('lat', 'lon') from data variable rhum as the horizontal dimensions for the regridding.
  warnings.warn(


#### Precipitation

In [4]:
Case_Study = False # If False, the whole European domain is downloaded 

CS_name = 'Crete' 

start_mid_year = False
home_path = "O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/unprocess/FWI-variables/"

if Case_Study == True:
    if CS_name == 'Crete':
        path = home_path + "Case_Studies/Crete/"
    else:
        print('CS not known')
else:
    path = home_path + "Europe/"

years = np.arange(1991, 2011, 1)

months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

for year in years:
    
    print(year)
    year = str(year)
 
    for m, month in enumerate(months): 
        print(month)
        fname = path + "/Derived-FWI-ERA5-land/vars-" + year + "-" + month + ".nc"

        # Only download files which are not in the folder
        if os.path.isfile("O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/regrided/FWI-variables/prcp-" + year + "-" + month + ".nc") == True:
            continue
            
        ds = xr.open_dataset(path + "ERA5-land/" + year + "-" + month + "-prcp.grib", engine = "cfgrib") # load the prcp at noon and midnight
        ds1 = xr.open_dataset(path  + "ERA5-land/" +  year + "-" + month + ".grib", engine = "cfgrib") # load the temp, dew temp and wind at noon
        
        ds1 = ds1.assign(prcp=(['time','latitude','longitude'], ds.isel(step = 1, time = slice(0, len(ds.time)-1)).tp.data)) # m to mm 
 
        #ds1 = ds1.assign(temp=lambda x: x.t2m - 273.15) # Convert K to C
 
        #ds1 = ds1.assign(wind=lambda x: np.sqrt(x.u10*x.u10 + x.v10*x.v10)*0.001*3600) # Calculate wind speed in km/h
   
        #ds1 = ds1.assign(rhum=lambda x: relative_humidity_from_dewpoint(x.temp* units.degC, (x.d2m- 273.15)* units.degC )*100) # Calculate relative humidity [%]

        ds1 = ds1.drop_vars(['u10', 'v10', 'd2m', 't2m', 'step'])
        ds1 = ds1.rename({'latitude': 'lat','longitude': 'lon'})
        ds1 = regridder(ds1, skipna=True, na_thres=0.5)
        ds1.to_netcdf("O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/regrided/FWI-variables/prcp-" + year + "-" + month + ".nc")


1991
01
02
03
04
05
06
07
08
09
10
11
12
1992
01
02
03
04
05
06
07
08
09
10
11
12
1993
01
02
03
04
05
06
07
08
09
10
11
12
1994
01
02
03
04
05
06
07
08
09
10
11
12
1995
01
02
03
04
05
06
07
08
09
10
11
12
1996
01
02
03
04
05
06
07
08
09
10
11
12
1997
01
02
03
04
05
06
07
08
09
10
11
12
1998
01
02
03
04
05
06
07
08
09
10
11
12
1999
01
02
03
04
05
06
07
08
09
10
11
12
2000
01
02
03
04
05
06
07
08
09
10
11
12
2001
01
02
03
04
05
06
07
08
09
10
11
12
2002
01
02
03
04
05
06
07
08
09
10
11
12
2003
01
02
03
04
05
06
07
08
09
10
11
12
2004
01
02
03
04
05
06
07
08
09
10
11
12
2005
01
02
03
04
05
06
07
08
09
10
11
12
2006
01
02
03
04
05
06
07
08
09
10
11
12
2007
01
02
03
04
05
06
07
08
09
10
11
12
2008
01
02
03
04
05
06
07
08
09
10
11
12
2009
01
02
03
04
05
06
07
08
09
10
11
12
2010
01
02
03
04
05
06
07
08
09
10
11
12


In [5]:
ds = xr.open_dataset(path + "ERA5-land/" + year + "-" + month + "-prcp.grib", engine = "cfgrib")

Ignoring index file 'O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/unprocess/FWI-variables/Europe/ERA5-land/2010-12-prcp.grib.5b7b6.idx' incompatible with GRIB file


In [9]:
ds.isel(step = 1).isel(time = slice(0, len(ds.time)-1))

<xarray.Dataset> Size: 42MB
Dimensions:     (time: 31, latitude: 451, longitude: 751)
Coordinates:
    number      int32 4B ...
  * time        (time) datetime64[ns] 248B 2010-11-30 2010-12-01 ... 2010-12-30
    step        timedelta64[ns] 8B 1 days
    surface     float64 8B ...
  * latitude    (latitude) float64 4kB 75.0 74.9 74.8 74.7 ... 30.2 30.1 30.0
  * longitude   (longitude) float64 6kB -30.0 -29.9 -29.8 ... 44.8 44.9 45.0
    valid_time  (time) datetime64[ns] 248B ...
Data variables:
    tp          (time, latitude, longitude) float32 42MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-26T13:23 GRIB to CDM+CF via cfgrib-0.9.1...